# Cython

## Loading the extension

IPtyhon notebook import Cython code. This extension can be loaded using the `pyximport` as follows:

In [1]:
import pyximport
pyximport.install(reload_support=True)
import fib_cython

## Using the Cython module

foo is the already compiling module function. 
Using foo module as follows.

In [2]:
fib_cython.__file__

'/Users/ohgushimasaya/.pyxbld/lib.macosx-10.9-x86_64-3.4/fib_cython.so.reload1'

In [3]:
fib_cython.fib(2)

1

Writing a normal function as follows.

In [4]:
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = a + b, a
    return a

In [5]:
fib(2)

1

In [6]:
import integral_max

In [7]:
dir(integral_max)

['__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__test__',
 'a_list',
 'call_allow',
 'generic_max']

In [8]:
 integral_max.generic_max(1,2)

array('d', [1.0, 2.0, 3.0])


2

多体問題シミュレーションのコード

元のコードは下記

https://github.com/cythonbook/examples

In [11]:
import sys 

def combinations(l):
    result = []
    for x in range(len(l) - 1):
        ls = l[x+1:]
        for y in ls:
            result.append((l[x],y))
    return result

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
        'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

        'jupiter': ([4.84143144246472090e+00,
            -1.16032004402742839e+00,
            -1.03622044471123109e-01],
            [1.66007664274403694e-03 * DAYS_PER_YEAR,
                7.69901118419740425e-03 * DAYS_PER_YEAR,
                -6.90460016972063023e-05 * DAYS_PER_YEAR],
            9.54791938424326609e-04 * SOLAR_MASS),

        'saturn': ([8.34336671824457987e+00,
            4.12479856412430479e+00,
            -4.03523417114321381e-01],
            [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
            2.85885980666130812e-04 * SOLAR_MASS),

        'uranus': ([1.28943695621391310e+01,
            -1.51111514016986312e+01,
            -2.23307578892655734e-01],
            [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
            4.36624404335156298e-05 * SOLAR_MASS),

        'neptune': ([1.53796971148509165e+01,
            -2.59193146099879641e+01,
            1.79258772950371181e-01],
            [2.68067772490389322e-03 * DAYS_PER_YEAR,
                1.62824170038242295e-03 * DAYS_PER_YEAR,
                -9.51592254519715870e-05 * DAYS_PER_YEAR],
            5.15138902046611451e-05 * SOLAR_MASS) 
        }


def advance(dt, n, bodies, pairs):

    for i in range(n):
        for (([x1, y1, z1], v1, m1),
                ([x2, y2, z2], v2, m2)) in pairs:
            dx = x1 - x2
            dy = y1 - y2
            dz = z1 - z2
            mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
            b1m = m1 * mag
            b2m = m2 * mag
            v1[0] -= dx * b2m
            v1[1] -= dy * b2m
            v1[2] -= dz * b2m
            v2[0] += dx * b1m
            v2[1] += dy * b1m
            v2[2] += dz * b1m
        for (r, [vx, vy, vz], m) in bodies:
            r[0] += dt * vx
            r[1] += dt * vy
            r[2] += dt * vz


def report_energy(bodies, pairs):

    e = 0.0

    for (((x1, y1, z1), v1, m1),
            ((x2, y2, z2), v2, m2)) in pairs:
        dx = x1 - x2
        dy = y1 - y2
        dz = z1 - z2
        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    for (r, [vx, vy, vz], m) in bodies:
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
    print("%.9f" % e)

def offset_momentum(ref, bodies):

    px = py = pz = 0.0

    for (r, [vx, vy, vz], m) in bodies:
        px -= vx * m
        py -= vy * m
        pz -= vz * m
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m

def main(n, bodies=BODIES, ref='sun'):
    system = list(bodies.values())
    pairs = combinations(system)
    offset_momentum(bodies[ref], system)
    report_energy(system, pairs)
    advance(0.01, n, system, pairs)
    report_energy(system, pairs)

if __name__ == '__main__':
    %timeit main(int(50000), bodies=BODIES, ref='sun')

-0.169075164
-0.169078071
-0.169304707
-0.169262077
-0.169171170
-0.169149750
-0.169036638
-0.169034805
1 loops, best of 3: 909 ms per loop


In [12]:
import nbody

In [13]:
%timeit nbody.main(int(50000), bodies=BODIES, ref='sun')

-0.169146771
-0.169149947
-0.169071064
-0.169072487
-0.169150511
-0.169169237
-0.169223879
-0.169253735
1 loops, best of 3: 403 ms per loop
